<a href="https://colab.research.google.com/github/ssaurabh5/Coursera_Capstone/blob/master/Capstone_Project_Notebook_for_the_battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursera Capstone Project on the battle of Neighborhoods

## About the city of Toronto

***Toronto, the capital of the province of Ontario, is a major Canadian city along Lake Ontario’s northwestern shore. It's a dynamic metropolis with a core of soaring skyscrapers, all dwarfed by the iconic, free-standing CN Tower. Toronto also has many green spaces, from the orderly oval of Queen’s Park to 400-acre High Park and its trails, sports facilities and zoo.***

Now we want to concentrate on the business problem that we want to solve and that is to find the most optimum location or neighborhood to set up a new Indian Restaurant in Toronto. Is it possible to predict the success of a new restaurant even before establishing it? Who would benefit from this Project?

* In this Project people who are looking to invest into Restaurants in Toronto region will be benefited.

**Let's see how?**


## The Steps taken for this project:


1. Business Understanding
2. Analytical Approach
3. Data Requirement 
4. Data Collection
5. Data Understanding
6. Data Preparation
7. Modeling
8. Conclusion or Results





## 1. Business Understanding
Before starting the project we will first define 
our goal and objective for the project.

**Goal** 
* To find the optimum location in Toronto to set up a new Indian Restaurant.

**Objective**
* To find the all the locations in Toronto
* Find the venues in the neighborhoods 
* Select one of the locations from the list having most venues and popualation density.

## 2. Analytical Approach
* The analytical approach would be clustering the locations as by clustering we will get the maximum number of density of venues and population

## 3. Data Requirements
* We will get our required data of toronto from internet and then from a a website we can pull the data in the form of a table

## 4. Data Collection
* We need to collect two types of datas one for the neighborhood and second for the coordinates of those neighborhoods


### Now we start to code and collect our data
* First we import the table from the website into our Dataframe in pandas

In [198]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # we defined our url here
df = pd.read_html(url)

In [199]:
df[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Then we select our table from various tables and select the null values from the table but here in our table null values are represented as 'Not assigned' so we define null values as 'Not assigned' for our dataframe.

In [200]:
df1 = pd.read_html(
    url, 
    na_values=["Not assigned"]
    )[0]

Now we are ready to drop or delete all the null values from our dataframe

In [201]:
df1 = df1.dropna()

Now to confirm our changes and to see the first 5 values of our dataframe

In [202]:
df1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


As we can see the index has dropped various values from our dataframe and the index numbers have rearranged so we reset the index to get a cleaner view of the dataframe

In [203]:
df2 = df1.reset_index(drop=True)

Here is our new dataframe with all the null values removed from it

In [204]:
df2.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [205]:
df2.shape

(103, 3)

Here we now take the file containing all the longitudes and latitudes and put it into a new dataframe

In [206]:
url2 = 'http://cocl.us/Geospatial_data'
df3 = pd.read_csv(url2)

After making a new dataframe we merge the new dataframe with the old one having all the information regarding Postal code as Postal code is common in both the dataframes.

In [207]:
df4 = df2.merge(df3)

Then we look at the first 13 rows in our new merged dataframe

In [208]:
df4.head(13)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Now we should check the shape of our new dataframe

In [209]:
df4.shape

(103, 5)

After getting all the data into the Dataframe it is time to import the remaining important libraries we require for further analysis

In [210]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium # uncomment this line if you haven't completed the Foursquare API lab

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library
import numpy as np
from geopy.geocoders import Nominatim
print('Libraries imported.')


Libraries imported.


Here we try to find out how many unique boroughs and neighborhoods we have

In [211]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df4['Borough'].unique()),
        df4.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


we define latitude and longitude from the internet of our desired area which is toronto here.

In [212]:
latitude = 43.651070
longitude= -79.347015

Now we can use folium library to show a beautiful map of toronto with blue labels showing the neighborhoods and bouroughs in Toronto.

In [213]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 5. Data Understanding
* In this step we will focus on which values should be selected and which values are to be ignored.

* Therefore, we get an idea of the total number of neighborhoods in that borough and from here we can see Downtown Toronto looks like an important area for toronto just after North york. For our analysis we will consider only Downtown toronto.

In [214]:
df4['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64

Now we only select Downtown Toronto from the Boroughs as we can see it is second biggest Borough and has the highest population density as it is located in the heart of the city.

In [215]:
toronto_data = df4[df4['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Now we give the Downtown Toronto dataframe to the map and form markers for the neighborhoods in downtown toronto.

In [216]:
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

## 6. Data Preparation
* Here we focus on getting the most useful data by adding,removing or replacing the data as required in the table.
* Now we go to the next step of using Foursquare to get all the venues around the Downtown Toronto region.

Now that we have our target Borough we will search for the most famous venues in that neighborhood using Foursquare API.

In [217]:
CLIENT_ID = 'HCSHKQ1FHXJMCCWF4NCYJMGUCRC41VRF3NUHMYI5R0HS1V2K' 
CLIENT_SECRET = 'NK31WIANWD3G1Z33HPKKDNGYADSE5L014V5S0KJ04Y4AWFDE' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HCSHKQ1FHXJMCCWF4NCYJMGUCRC41VRF3NUHMYI5R0HS1V2K
CLIENT_SECRET:NK31WIANWD3G1Z33HPKKDNGYADSE5L014V5S0KJ04Y4AWFDE


We want to test our code for venues in 1 neighborhood, as it gives the result faster than giving it the whole dataframe to search venues

In [218]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

We derive neighborhood name,latitude and longitude for that neighborhood.

In [219]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Now we can give all the required data to Foursquare to find the venues in the neighborhood.

In [220]:
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HCSHKQ1FHXJMCCWF4NCYJMGUCRC41VRF3NUHMYI5R0HS1V2K&client_secret=NK31WIANWD3G1Z33HPKKDNGYADSE5L014V5S0KJ04Y4AWFDE&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

We get all the data from Foursquare in the form of json

In [221]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3ae53231e6642aa633cff7'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 143,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

Now we start to build function to get categories from venue

In [222]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

We complete our code with nearby venues in a dataframe

In [223]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [224]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))  #We just count the number of venues here

43 venues were returned by Foursquare.


As our test code was successful to get all the venues for 1 neighborhood, we now do the same for all the neighborhoods in Downtown Toronto.

In [225]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We now use the function to get the nearby values

In [226]:
toronto_data_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


## 7. Modeling and Clustering the Neighborhoods of Toronto
* Now we will be fitting all our data into the K-means clustering model to get the clusters of our neighborhood.

In [227]:
print(toronto_data_venues.shape)  #We see here the shape of our dataframe

(1239, 7)


Now we want to Group our Data by Nighborhoods, so we get the combined values of venues in those neighborhoods.

In [228]:
toronto_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [229]:
print('There are {} uniques categories.'.format(len(toronto_data_venues['Venue Category'].unique())))   #We find the unique categories of venues

There are 210 uniques categories.


One hot encoder is used for machine to process the data faster for machine learning activities and converts all the data into numerical values.

In [230]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,...,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [231]:
toronto_onehot.shape

(1239, 210)

Now we group our 

In [232]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,...,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.00,0.017544,0.00,0.000000,0.00,0.000000,0.000000,...,0.035088,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.035088,0.000000,0.017544,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.017544,0.000000,0.000000,0.017544,0.00,0.017544,0.000000,0.00,0.000000,0.017544,0.000000,0.000000,0.00,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.066667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,Central Bay Street,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.031250,0.00,0.031250,0.000000,...,0.000000,0.00,0.000000,0.03125,0.000000,0.046875,0.00,0.000000,0.015625,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.015625,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.015625,0.00,0.000000,0.000000,0.00,0.015625,0.015625,0.000000,0.000000,0.00,0.00,0.00,0.015625,0.000000,0.000000,0.015625,0.00
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,...,0.058824,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,Church and Wellesley,0.026316,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.00,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.013

In [233]:
toronto_grouped.shape

(19, 210)

Now we are trying to find the number of shops or frequency of shops in a particular neighborhood.

In [234]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1        Beer Bar  0.04
2    Cocktail Bar  0.04
3  Farmers Market  0.04
4      Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4          Boutique  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.05
2      Sandwich Place  0.05
3  Italian Restaurant  0.05
4    Department Store  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3          Diner  0.06
4    Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.11
1  Japanese Restaurant  0.07
2     Sushi Restaurant  0.05
3              Gay Bar  0.04
4           Restaurant  0.04


--

We make a function to return most common venues

In [235]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Using the previous function we find the 10 most common venues in all the neighborhoods of toronto.

In [236]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(13)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Seafood Restaurant,Beer Bar,Cocktail Bar,Farmers Market,Café,Cheese Shop,Japanese Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden,Plane,Boutique
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Ice Cream Shop
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Restaurant,Candy Store,Diner,Baby Store,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Men's Store,Café,Bubble Tea Shop,Pub
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Seafood Restaurant,Asian Restaurant,Deli / Bodega,Bar
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,American Restaurant,Steakhouse,Seafood Restaurant,Asian Restaurant
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Hotel,Fast Food Restaurant,Bookstore
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Fried Chicken Joint,Restaurant,Brewery,Italian Restaurant,Park
9,"Kensington Market, Chinatown, Grange Park",Coffee Shop,Vegetarian / Vegan Restaurant,Café,Vietnamese Restaurant,Mexican Restaurant,Burger Joint,Bar,Caribbean Restaurant,Dumpling Restaurant,Bakery


We cluster all the neighborhoods into 5 clusters using K-Means Clustering

In [237]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:10]

array([4, 2, 4, 3, 4, 4, 4, 1, 4, 1], dtype=int32)

In [238]:
#neighborhoods_venues_sorted.drop('Cluster Labels',axis=1,inplace=True)

Adding the Cluster Label to our original dataframe so we can cluster our neighnorhoods clearly.

In [239]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head(13) # check the last columns!


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Mexican Restaurant,Restaurant,Chocolate Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Yoga Studio,College Cafeteria,Smoothie Shop,Beer Bar,Sandwich Place,Portuguese Restaurant,Café,Persian Restaurant,College Auditorium
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Hotel,Fast Food Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Cosmetics Shop,Restaurant,Cocktail Bar,American Restaurant,Clothing Store,Park,Moroccan Restaurant,Creperie
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Restaurant,Bakery,Seafood Restaurant,Beer Bar,Cocktail Bar,Farmers Market,Café,Cheese Shop,Japanese Restaurant
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Ice Cream Shop
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Restaurant,Candy Store,Diner,Baby Store,Nightclub
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,4,Coffee Shop,Café,Clothing Store,Restaurant,Gym,Hotel,Steakhouse,Thai Restaurant,Bar,Office
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,4,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Fried Chicken Joint,Restaurant,Brewery,Italian Restaurant,Park
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,4,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Salad Place,Seafood Restaurant,Japanese Restaurant,American Restaurant,Bar


Now using Folium again we can now see the clusters that we have got from our clustering method

In [240]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

From here on it is all about the analysis of venues in various clusters.

In [241]:
Cluster1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,0,Park,Trail,Playground,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [242]:
Cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Hotel,Fast Food Restaurant,Bookstore
3,Downtown Toronto,1,Café,Coffee Shop,Cosmetics Shop,Restaurant,Cocktail Bar,American Restaurant,Clothing Store,Park,Moroccan Restaurant,Creperie
11,Downtown Toronto,1,Café,Restaurant,Bar,Japanese Restaurant,Sandwich Place,Bookstore,Bakery,Yoga Studio,Italian Restaurant,Beer Bar
12,Downtown Toronto,1,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Vietnamese Restaurant,Mexican Restaurant,Burger Joint,Bar,Caribbean Restaurant,Dumpling Restaurant,Bakery
16,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Chinese Restaurant,Bakery,Pub,Pizza Place,General Entertainment,Butcher


In [243]:
Cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,2,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden,Plane,Boutique


In [244]:
Cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,3,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Restaurant,Candy Store,Diner,Baby Store,Nightclub


In [245]:
Cluster5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Mexican Restaurant,Restaurant,Chocolate Shop
1,Downtown Toronto,4,Coffee Shop,Yoga Studio,College Cafeteria,Smoothie Shop,Beer Bar,Sandwich Place,Portuguese Restaurant,Café,Persian Restaurant,College Auditorium
4,Downtown Toronto,4,Coffee Shop,Restaurant,Bakery,Seafood Restaurant,Beer Bar,Cocktail Bar,Farmers Market,Café,Cheese Shop,Japanese Restaurant
5,Downtown Toronto,4,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Ice Cream Shop
7,Downtown Toronto,4,Coffee Shop,Café,Clothing Store,Restaurant,Gym,Hotel,Steakhouse,Thai Restaurant,Bar,Office
8,Downtown Toronto,4,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Fried Chicken Joint,Restaurant,Brewery,Italian Restaurant,Park
9,Downtown Toronto,4,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Salad Place,Seafood Restaurant,Japanese Restaurant,American Restaurant,Bar
10,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Seafood Restaurant,Asian Restaurant,Deli / Bodega,Bar
15,Downtown Toronto,4,Coffee Shop,Café,Pub,Italian Restaurant,Restaurant,Beer Bar,Seafood Restaurant,Gym,Hotel,Japanese Restaurant
17,Downtown Toronto,4,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,American Restaurant,Steakhouse,Seafood Restaurant,Asian Restaurant


Here we can see Clearly that what venues are most popular in each cluster and now we will investigate which neighborhood is optimum to set a new Indian Restaurant.

In [246]:
df6 = toronto_merged.sort_values(by='Cluster Labels',ignore_index=True)
df6

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Trail,Playground,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Chinese Restaurant,Bakery,Pub,Pizza Place,General Entertainment,Butcher
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Hotel,Fast Food Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Cosmetics Shop,Restaurant,Cocktail Bar,American Restaurant,Clothing Store,Park,Moroccan Restaurant,Creperie
4,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,1,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Vietnamese Restaurant,Mexican Restaurant,Burger Joint,Bar,Caribbean Restaurant,Dumpling Restaurant,Bakery
5,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,1,Café,Restaurant,Bar,Japanese Restaurant,Sandwich Place,Bookstore,Bakery,Yoga Studio,Italian Restaurant,Beer Bar
6,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,2,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden,Plane,Boutique
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Restaurant,Candy Store,Diner,Baby Store,Nightclub
8,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Mexican Restaurant,Restaurant,Chocolate Shop
9,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,4,Coffee Shop,Café,Pub,Italian Restaurant,Restaurant,Beer Bar,Seafood Restaurant,Gym,Hotel,Japanese Restaurant


## 8. Conclusion or Results
* We can say that cluster 2 and cluster 5 are the ones with the highest number of venues but cluster 5 has more coffee shops than restaurants so we will set up an Indian restaurant in cluster 5 as cluster 5 has more population density and the frequency of restaurants is lower than coffee shops.
* In cluster 2 we can see that even though coffee shop and café are most common but there are Restaurants as well compared to the cluster 5 where frequency of Restaurants was not so prominent.


*So we will select **Central Bay Street** from cluster 5 as our neighbourhood of choice as it has less frequency of Restaurants and similarly we can select more locations from cluster 5 where there are, no Restaurants as top 3 venues.*